### Training notebook
- https://www.kaggle.com/code/ravaghi/skin-cancer-detection-resnet50-training

In [1]:
from torch.utils.data import DataLoader, Dataset
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
import albumentations as A
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import pandas as pd
import numpy as np
import warnings
import h5py
import torch
import random
import timm
import cv2
import io
import os
import gc

warnings.filterwarnings('ignore')

In [2]:
class CFG:
    test_metadata_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv'
    test_img_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
    sample_sub_path = '/kaggle/input/isic-2024-challenge/sample_submission.csv'
    checkpoint_path = '/kaggle/input/skin-cancer-detection-resnet50-training/checkpoints/'

    checkpoints = [
        'fold_0_epoch_10_pauc_0.119877.pth',
        'fold_1_epoch_16_pauc_0.152112.pth',
        'fold_2_epoch_11_pauc_0.146086.pth',
        'fold_3_epoch_20_pauc_0.161103.pth',
        'fold_4_epoch_48_pauc_0.169901.pth'
    ]

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    val_batch_size = 64
    image_size  = 224
    n_folds = 5
    seed = 35555
    model_name = "resnet50"

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(CFG.seed)

In [4]:
test_metadata = pd.read_csv(CFG.test_metadata_path)

In [5]:
class ISICDataset(Dataset):
    def __init__(self, data_path, metadata, transform=None):
        self.data = h5py.File(data_path, 'r')
        self.metadata = metadata
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = self.metadata.iloc[idx]['isic_id']
        image = np.array(self.data[img_name])
        image = np.array(Image.open(io.BytesIO(image)))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        if 'target' in self.metadata.columns:
            label = int(self.metadata['target'].iloc[idx])
            return image, label
        else:
            return image, 0

In [6]:
transforms = A.Compose([
    A.Resize(CFG.image_size, CFG.image_size),
    A.Normalize(
        mean=[0.4815, 0.4578, 0.4082],
        std=[0.2686, 0.2613, 0.2758],
        max_pixel_value=255.0),
    ToTensorV2(),
])

In [7]:
class ResNet(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super(ResNet, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=3)
        num_features = 1000
        self.classifier = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        x = self.classifier(x)
        return x

In [8]:
class Trainer:
    def __init__(self, device, model, test_dataloader, checkpoint_path):
        self.device = device
        self.model = model
        self.test_dataloader = test_dataloader
        self.checkpoint_path = checkpoint_path

    def predict(self):
        self.model.load_state_dict(torch.load(self.checkpoint_path))
        self.model.eval()

        preds = []
        with torch.no_grad():
            loop = tqdm(self.test_dataloader, total=len(self.test_dataloader))
            for batch in loop:
                x, _ = batch
                x = x.to(self.device)
                y_hat = self.model(x).squeeze()
                preds.extend(y_hat.detach().cpu().numpy())
                loop.set_description(f'Prediction')

        return preds

In [9]:
test_preds = []
for fold_idx in range(CFG.n_folds):

    test_dataset = ISICDataset(CFG.test_img_path, test_metadata, transforms)
    test_dataloader = DataLoader(test_dataset, batch_size=CFG.val_batch_size, shuffle=False, num_workers=4)

    model = ResNet(CFG.model_name).to(CFG.device)

    trainer = Trainer(
        device=CFG.device,
        model=model,
        test_dataloader=test_dataloader,
        checkpoint_path=CFG.checkpoint_path + CFG.checkpoints[fold_idx]
    )

    preds = trainer.predict()
    test_preds.append(preds)

    del test_dataset, test_dataloader, model, trainer
    gc.collect()
    torch.cuda.empty_cache()

Prediction: 100%|██████████| 1/1 [00:00<00:00,  7.51it/s]


In [10]:
sub = test_metadata.copy()
sub['target'] = np.mean(test_preds, axis=0)
sub = sub[['isic_id', 'target']]
sub.to_csv('submission.csv', index=False)
sub.head()

,isic_id,target
0,ISIC_0015657,0.011530
1,ISIC_0015729,0.001099
2,ISIC_0015740,0.002055
